Handle Imports

In [1]:
import sys
import os
import ast
import argparse
import pandas as pd
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
from logging_plotter import parse_datastamp_topic_csv


MACROS

In [2]:
TEST_FOLDER = '/home/will/drl_swarm/data_analysis/rotation_matrix_testing/flipped_sensor_pos_config/90_pitch_90_yaw/'
CF_VICON_FILE = TEST_FOLDER + 'cf_vicon.csv'
TB1_VICON_FILE = TEST_FOLDER + 'tb1_vicon.csv'
TB2_VICON_FILE = TEST_FOLDER + 'tb2_vicon.csv'
LH_FILE = TEST_FOLDER + 'lh.csv'
ODOM_FILE = TEST_FOLDER + 'odom.csv'

 Read in vicon outputs
 
 **NOTE: Make sure the topics line up with the data you're analyzing**

In [3]:
cf_vicon_topics= {
                "/rigid_bodies.rigidbodies[0].pose.position.x": "cf_x",
                "/rigid_bodies.rigidbodies[0].pose.position.y": "cf_y",
                "/rigid_bodies.rigidbodies[0].pose.position.z": "cf_z"
                }
tb1_topics = {
                "/rigid_bodies.rigidbodies[1].pose.position.x": "tb1_x",
                "/rigid_bodies.rigidbodies[1].pose.position.y": "tb1_y",
                "/rigid_bodies.rigidbodies[1].pose.position.z": "tb1_z"
                }
tb2_topics = {
                "/rigid_bodies.rigidbodies[2].pose.position.x": "tb2_x",
                "/rigid_bodies.rigidbodies[2].pose.position.y": "tb2_y",
                "/rigid_bodies.rigidbodies[2].pose.position.z": "tb2_z"
                }

odom_topics = {"/odom.pose.pose.position.x": "odom_x",
                "/odom.pose.pose.position.y": "odom_y",
                "/odom.pose.pose.position.z": "odom_z",}
lh_topics = {'/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.x': 'lh_x',
                '/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.y': 'lh_y',
                '/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.z': 'lh_z',}

In [4]:
cf_vicon_data = parse_datastamp_topic_csv(CF_VICON_FILE, cf_vicon_topics)
tb1_vicon = parse_datastamp_topic_csv(TB1_VICON_FILE, tb1_topics)
tb2_vicon = parse_datastamp_topic_csv(TB2_VICON_FILE, tb2_topics)
df_lh = parse_datastamp_topic_csv(LH_FILE, lh_topics)
df_odom = parse_datastamp_topic_csv(ODOM_FILE, odom_topics)
tb_vicon_data = pd.concat([tb1_vicon, tb2_vicon], axis=1)


Simple Plot Outputs

In [5]:
fig1 = tb_vicon_data.plot(title="TB Vicon Data", x =tb_vicon_data.index, y=tb_vicon_data.columns, kind = 'scatter', template = 'plotly_dark')
fig1.show(renderer="browser")

fig2 = cf_vicon_data.plot(title="CF Vicon Data", x =cf_vicon_data.index, y=cf_vicon_data.columns, kind = 'scatter', template = 'plotly_dark')
fig2.show(renderer="browser")

fig3 = df_lh.plot(title="Lighthouse Data", x =df_lh.index, y=df_lh.columns, kind = 'scatter', template = 'plotly_dark')
fig3.show(renderer="browser")

fig4 = df_odom.plot(title="Odometry Data", x =df_odom.index, y=df_odom.columns, kind = 'scatter', template = 'plotly_dark')
fig4.show(renderer="browser")


/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Opening in existing browser session.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence t

Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so


# Start of Data Analysis and Cleaning

### Initial Positions

Get offsets for crazyflie lh and vicon initial positions

In [6]:
vicon_cf_offset = (cf_vicon_data.dropna()).head(75).mean()
lh_cf_offset = (df_lh.dropna()).head(75).mean()
print(f"vicon_cf_offset: {vicon_cf_offset}")
print(f"lh_cf_offset: {lh_cf_offset}")

vicon_cf_offset: topic
cf_x    0.526854
cf_y    0.077240
cf_z    0.986290
dtype: float64
lh_cf_offset: topic
lh_x    0.620721
lh_y    0.048385
lh_z    0.163604
dtype: float64


Get offsets for the turtlebot positions

In [7]:
print(tb_vicon_data.mean())

topic
tb1_x    0.859986
tb1_y   -0.442818
tb1_z    0.177401
tb2_x    0.857588
tb2_y    0.629936
tb2_z    0.185581
dtype: float64


## Compare Crazyflie position when computed from Vicon and Lighthouse

In [8]:
comparison_df = pd.concat([cf_vicon_data, df_lh], axis=1)
comparison_df.dropna(axis=0, how='all', inplace=True)
fig5 = comparison_df.plot(title="Comparison plot", x = comparison_df.index, y = comparison_df.columns, kind = "scatter", template="plotly_dark")
fig5.show(renderer="browser")

/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Opening in existing browser session.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Subtract initial position from `comparison_df`

In [9]:
comparison_df_normalized = comparison_df - pd.concat([vicon_cf_offset, lh_cf_offset])
comparison_df_x = comparison_df_normalized.filter(like="_x")
comparison_df_y = comparison_df_normalized.filter(like="_y")
comparison_df_z = comparison_df_normalized.filter(like="_z")

In [10]:
fig6 = comparison_df_x.plot(title="Vicon vs LH x", x = comparison_df_x.index, y = comparison_df_x.columns, kind = "scatter", template="plotly_dark")
fig6.show(renderer="browser")

fig7 = comparison_df_y.plot(title="Vicon vs LH y", x = comparison_df_y.index, y = comparison_df_y.columns, kind = "scatter", template="plotly_dark")
fig7.show(renderer="browser")

fig8 = comparison_df_z.plot(title="Vicon vs LH z", x = comparison_df_z.index, y = comparison_df_z.columns, kind = "scatter", template="plotly_dark")
fig8.show(renderer="browser")


/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Opening in existing browser session.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/home/will/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Opening in existing browser session.
Opening in existing browser session.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/will/snap/code/common/.cache/gio-modules/libgiolibproxy.so
